In [2]:
import warnings
from math import*
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import time
import pickle
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import scipy.io as scio
from scipy.stats import norm
import statsmodels.api as sm

1. 用冪法求下列矩陣的主特徵值及其特徵向量。注意：需要輸出每一步迭代。

請寫出Python程序，文件名為

In [3]:
def Power_Method_EigenValueVector(A,E):
    n=len(A)
    z0=np.random.random((n,1))
    lamda0=0.9
    lamda1=1.0
    while   abs(lamda1-lamda0)>E:
        lamda0=lamda1
        w=np.dot(A,z0)
        absw=abs(w)
        i_maxabsw=np.argmax(absw)#最大值索引
        z1=w/w[i_maxabsw]
        lamda1=w[0]/z0[0]#特征值
        z0=z1
        # print(lamda1)
        # print(z1)
    eigvalue=lamda1#特征值
    eigvector=z1#特征向量
    return  eigvalue,eigvector

A=np.array([[3,-4,3],[-4,6,3],[3,10,1]])
E=10**(-3)
eigvalue,eigvector=Power_Method_EigenValueVector(A,E)
print(eigvalue)
print(eigvector)

[9.73192039]
[[-0.08894735]
 [ 0.89967382]
 [ 1.        ]]


2. 請從銳思資料庫下載000568, 000025, 000543, 600870, 600639, 000556, 600694, 600808, 600601, 600630,606,0606 63在2015年1月1日至2019年12月31日日的日用數據，計算任兩檔股票的相關係數，構成相關係數矩陣，並以冪法就相關係數矩陣的最大特徵對應其特徵表示。

          0         1         2         3         4         5         6   \
0   1.000000  0.146236  0.354403  0.268390  0.100218  0.100047  0.380392   
1   0.146236  1.000000  0.119014  0.252200  0.052211  0.179165  0.159010   
2   0.354403  0.119014  1.000000  0.344586  0.243826  0.246808  0.400007   
3   0.268390  0.252200  0.344586  1.000000  0.196340  0.344174  0.369017   
4   0.100218  0.052211  0.243826  0.196340  1.000000  0.158871  0.126296   
5   0.100047  0.179165  0.246808  0.344174  0.158871  1.000000  0.297860   
6   0.380392  0.159010  0.400007  0.369017  0.126296  0.297860  1.000000   
7   0.421618  0.198605  0.441251  0.491729  0.203185  0.221852  0.430956   
8   0.307587  0.180180  0.299992  0.334843  0.165502  0.167027  0.407854   
9   0.230718  0.133983  0.303828  0.263628  0.290419  0.196290  0.336466   
10  0.081288  0.167987  0.187570  0.213036  0.446118  0.215814  0.159203   
11  0.052506  0.126468  0.124599  0.272686  0.289865  0.201031  0.175038   
12  0.214861

3. 根據沉櫻櫻等人的研究 [Y.-Y. Shen, Z.-Q. Jiang, J.-C. Ma, G.-J. Wang, W.-X. Zhou, Sector connectivity in the Chinese stocks markets, Empirical Economics, 2021]，中美貿易戰期間，中國股市行業間的風險傳導網絡如下所示：

研究表明特徵向量中心性是度量複雜網絡中節點影響力有效方法，特徵向量中心性即為網絡鄰接矩陣最大特徵值對應的特徵向量。請利用所給的行業風險傳導關係數據構建鄰接矩陣，並結合冪法識別出風險傳導網絡中影響力排名前十的行業。
鄰接矩陣計算提示：若已知A，B，C，D表示4個行業，它們之間的有向連接關係為


In [10]:

# 載入數據
data = pd.read_csv('20211207-Python-上机数据11_RankSectors.csv')
data.columns = ["from_sector", "to_sector", "weight"]

# 提取唯一的目標行業
X1 = data['to_sector'].unique()

# 初始化權重的 DataFrame
datan = pd.DataFrame(0, index=X1, columns=X1)

# 填充 DataFrame 中的權重
for i in X1:
    for j in X1:
        # 根據 from_sector 和 to_sector 找到對應的權重
        weight_value = data[(data['from_sector'] == j) & (data['to_sector'] == i)]['weight']
        datan.loc[i, j] = weight_value.values[0] if not weight_value.empty else 0

def Power_Method_EigenValueVector(A, E):
    n = len(A)
    z0 = np.random.random((n, 1))  # 隨機初始化特徵向量
    lambda0 = 0.9
    lambda1 = 1
    # 迭代直到特徵值收斂
    while abs(lambda1 - lambda0) > E:
        lambda0 = lambda1
        w = np.dot(A, z0)  # 計算 A*z0
        absw = abs(w)
        i_maxabsw = np.argmax(absw)  # 找到最大絕對值的索引
        z1 = w / w[i_maxabsw]  # 正規化特徵向量
        lambda1 = w[0] / z0[0]  # 計算新的特徵值
        z0 = z1

    eigvalue = lambda1
    eigvector = z1
    return eigvalue, eigvector

# 將 DataFrame 轉換為 numpy 陣列
A = datan.values
E = 10**-3  # 收斂精度
eigvalue, eigvector = Power_Method_EigenValueVector(A, E)

print("最大特徵值為：", eigvalue)
print("對應的特徵向量為：", eigvector)

# 找出前 10 名行業
shunxu = np.sort(eigvector, axis=0)[-10]
print("前10名為：")
for i in range(len(eigvector)):
    if eigvector[i][0] >= shunxu:
        print(X1[i])

# 準備建立結果矩陣
aset = set(data["from_sector"])  # 獨特的來源行業集合
bset = set(data["to_sector"])  # 獨特的目標行業集合
N = len(aset)  # 來源行業的數量
N1 = len(bset)  # 目標行業的數量

# 初始化結果矩陣的 DataFrame
dfSheet1 = pd.DataFrame(0, index=list(aset), columns=list(bset))

# 填充矩陣的函數
def toMatrix():
    for i in range(len(data)):
        from_sector = data.loc[i, "from_sector"]
        to_sector = data.loc[i, "to_sector"]
        weight = data.loc[i, "weight"]
        # 確保來源和目標行業在矩陣索引和列中存在
        if from_sector in dfSheet1.index and to_sector in dfSheet1.columns:
            dfSheet1.loc[from_sector, to_sector] = weight  # 設置權重
    return dfSheet1

# 調用函數並打印矩陣
result_matrix = toMatrix()
print(result_matrix)

最大特徵值為： [2.89394308]
對應的特徵向量為： [[0.46819088]
 [0.34582222]
 [0.5748515 ]
 [0.4937695 ]
 [0.19602796]
 [0.55523881]
 [0.95726675]
 [1.        ]
 [0.74008759]
 [0.68359799]
 [0.62770381]
 [0.68215144]
 [0.47642239]
 [0.43660265]
 [0.56614708]
 [0.18203723]
 [0.94422746]
 [0.65555735]
 [0.23654366]
 [0.51580367]
 [0.30077088]
 [0.51505851]
 [0.5426397 ]
 [0.8067324 ]
 [0.16701338]
 [0.58206044]
 [0.67273935]
 [0.6611275 ]]
前10名為：
HApp
F&D
T&A
LMF
Util
Cps
BM
Comm
Auto
MechE
             HApp      ComT        BM      Comm        ND       B&D      Util  \
HApp     0.000000  0.000000  0.302764  0.000000  0.000000  0.000000  0.204602   
ComT     0.498456  0.000000  0.225970  0.301319  0.000000  0.000000  0.186901   
BM       0.267091  0.000000  0.000000  0.000000  0.237667  0.000000  0.000000   
Comm     0.000000  0.554245  0.582507  0.000000  0.497014  0.302576  0.740045   
ND       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Util     0.129234  0.000000  0.000000 